In [9]:
import json
import requests
import time


# Load the boxscore JSON file
with open('assets/json/boxscores/mlb_boxscores_2025-07-05.json', 'r') as f:
    all_boxscores = json.load(f)

all_ids = set()

for game in all_boxscores:
    boxscore = game.get('boxscore', {})
    teams = boxscore.get('teams', {})
    for team_type in ['home', 'away']:
        team = teams.get(team_type, {})
        for key in ['batters', 'pitchers', 'bench']:
            for pid in team.get(key, []):
                all_ids.add(pid)

player_info = []

for pid in sorted(all_ids):
    url = f"https://statsapi.mlb.com/api/v1/people/{pid}"
    try:
        resp = requests.get(url)
        if resp.status_code == 200:
            data = resp.json()
            if data.get("people"):
                player_info.append(data["people"][0])
        else:
            print(f"Failed to fetch info for player ID {pid}")
    except Exception as e:
        print(f"Error fetching player {pid}: {e}")
    time.sleep(0.1)  # Be polite to the API

print("Fetched player info for", len(player_info), "players")

Fetched player info for 527 players


In [4]:
print(player_info[0])  # Print first player's info for verification 

{'id': 445276, 'fullName': 'Kenley Jansen', 'link': '/api/v1/people/445276', 'firstName': 'Kenley', 'lastName': 'Jansen', 'primaryNumber': '74', 'birthDate': '1987-09-30', 'currentAge': 37, 'birthCity': 'Willemstad', 'birthCountry': 'Curacao', 'height': '6\' 5"', 'weight': 265, 'active': True, 'primaryPosition': {'code': '1', 'name': 'Pitcher', 'type': 'Pitcher', 'abbreviation': 'P'}, 'useName': 'Kenley', 'useLastName': 'Jansen', 'middleName': 'Geronimo', 'boxscoreName': 'Jansen, K', 'nickName': 'Kenleyfornia', 'gender': 'M', 'isPlayer': True, 'isVerified': True, 'pronunciation': 'KEHN-lee JAN-sin', 'mlbDebutDate': '2010-07-24', 'batSide': {'code': 'S', 'description': 'Switch'}, 'pitchHand': {'code': 'R', 'description': 'Right'}, 'nameFirstLast': 'Kenley Jansen', 'nameSlug': 'kenley-jansen-445276', 'firstLastName': 'Kenley Jansen', 'lastFirstName': 'Jansen, Kenley', 'lastInitName': 'Jansen, K', 'initLastName': 'K Jansen', 'fullFMLName': 'Kenley Geronimo Jansen', 'fullLFMName': 'Jansen,

In [7]:
with open('assets/json/players/player_info.json', 'w', encoding='utf-8') as f_out:
    json.dump(player_info, f_out, ensure_ascii=False, indent=2)
print("player_info.json file created with", len(player_info), "players")

player_info.json file created with 527 players


In [11]:
import pandas as pd
from pandas import json_normalize

In [12]:
# Flatten nested fields
df = json_normalize(player_info)

# Write to CSV
df.to_csv('assets/csv/players/player_info.csv', index=False)